### Preparar ambiente

In [1]:
import enum
print(enum.__file__)  
# standard library location should be something like 
# /usr/local/lib/python3.6/enum.py

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/enum.py


In [2]:
import sys
!{sys.executable} -m pip uninstall -y enum34

In [3]:
!{sys.executable} -m pip install awswrangler

In [4]:
!{sys.executable} -m pip install geopandas

In [5]:
!{sys.executable} -m pip install PyAthena

### Conectar con pyathena

In [6]:
from pyathena.pandas.cursor import PandasCursor
from pyathena import connect
import pandas as pd
import time
# agregar directorio en el cual se almacenan las tablas que se van a consultar
directorio = 's3://iadbprod-csd-hub-analyticaldata/graphdata-mobility-temporal/athena-results/'
# base de datos
bd = 'graphdata' # Database in Glue
cursor = connect(s3_staging_dir = directorio, region_name = 'us-east-1', schema_name = bd, cursor_class = PandasCursor).cursor()

In [7]:
import pandas as pd
import awswrangler as wr
import numpy as np
import calendar
import boto3
s3 = boto3.resource('s3')

In [8]:
import geopandas as gpd
import glob
import os

### Verificar que están localmente los archivos Shapefile. Si no están, descargarlos

In [9]:
if not os.path.exists('Shapefiles_mun'):
    bucket_shapefiles = 'iadbprod-csd-hub-analyticaldata'
    folder_local = 'Shapefiles_mun'
    os.mkdir(folder_local)
    for key in s3.Bucket(bucket_shapefiles).objects.all().filter(Prefix=folder_local):
        data_key = key.key
        directorios = data_key.split('/')
        if len(directorios) == 3 and not 'Municipios' in data_key: # carpeta originen, pais y archivo (shp, cpg, csv, dbf, prj, shx)
            folder_pais = os.path.join(folder_local, directorios[1])
            if not os.path.exists(folder_pais):
                os.mkdir(folder_pais)
            new_data_key = os.path.join(folder_pais,directorios[2])
            s3.Bucket(bucket_shapefiles).download_file(data_key, new_data_key)

### Asignar nueva region a usuarios

In [10]:
pais = 'BR'

In [11]:
query = f'''SELECT iso_country_code,
                   id_provincia,
                   lat_final, 
                   long_final
            FROM historico_usuarios                             
            WHERE iso_country_code='{pais}'
            GROUP BY iso_country_code,
                     id_provincia,
                     lat_final, 
                    long_final'''
df = cursor.execute(query).as_pandas()    
print(f"Tamaño tabla: {df.shape[0]}")
assert df[df.duplicated(['long_final','lat_final','iso_country_code', 'id_provincia'])].shape[0] == 0
gdf_homes = gpd.GeoDataFrame(df,geometry = gpd.points_from_xy(df.long_final, df.lat_final), crs = 'epsg:4326')
gdf_homes = gdf_homes.dropna(subset = ["long_final",'lat_final'])

Tamaño tabla: 5542991


In [12]:
gdf_homes.shape

(5542990, 5)

In [13]:
# adm4 - Municipios
# id_adm = 'cd_geocmu'
loc_adm =f"Shapefiles_mun/{pais}/*.shp"
path = glob.glob(f'{loc_adm}')
mapa = gpd.read_file(path[0], encoding = 'utf-8')
mapa['geometry'] = mapa['geometry'].to_crs(epsg = 4326)
mapa.columns = [c.lower() for c in mapa.columns]
gdf_homes.crs = mapa.crs
time.sleep(10)
mapa = mapa.loc[:,['codmunic','nommunic','cd_geocmi', 'geometry']]
gdf_homes_joined = gpd.sjoin(gdf_homes, mapa, op = 'within') 
gdf_homes_joined = gdf_homes_joined[['iso_country_code', 'lat_final', 'long_final', 'codmunic','nommunic','cd_geocmi']]
gdf_homes_joined['lat_final'] = round(gdf_homes_joined['lat_final'], 3)
gdf_homes_joined['long_final'] = round(gdf_homes_joined['long_final'], 3)
gdf_homes = gdf_homes_joined

In [14]:
gdf_homes.shape

(5520829, 6)

In [15]:
table_uc = f'historico_latlong_{pais}'
wr.s3.to_parquet(  # Storing the data and metadata to Data Lake
    df = gdf_homes,
    dataset = True,
    table = table_uc,
    database = "graphdata",
    path = f"s3://iadbprod-csd-hub-analyticaldata/graphdata-mobility-usuarios/Historico_LatLong/{pais}/{table_uc}"
)

{'paths': ['s3://iadbprod-csd-hub-analyticaldata/graphdata-mobility-usuarios/Historico_LatLong/BR/historico_latlong_BR/01c2a9a940974a3ab847b2bcef28e46c.snappy.parquet'],
 'partitions_values': {}}